In [19]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
dataset = {'Physics':['Low','Med','Med','Med','Med','High','Low','High','Med','Low'],
       'Chemistry':['Low','Low','Low','High','Low','High','High','Low','High','Low'],
       'Maths':['Low','Low','High','High','High','Low','Low','Low','Low','High'],
       'Pass':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}
df = pd.DataFrame(dataset,columns=['Physics','Chemistry','Maths','Pass'])
entropy_node = 0  
values = df.Pass.unique() 
for value in values:
    fraction = df.Pass.value_counts()[value]/len(df.Pass)
    entropy_node += -fraction*np.log2(fraction)
attribute = 'Physics'
target_variables = df.Pass.unique() 
variables = df[attribute].unique()   
entropy_attribute = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        num = len(df[attribute][df[attribute]==variable][df.Pass ==target_variable]) 
        den = len(df[attribute][df[attribute]==variable])  
        fraction = num/(den+eps)  
        entropy_each_feature += -fraction*log(fraction+eps) 
    fraction2 = den/len(df)
    entropy_attribute += -fraction2*entropy_each_feature   
def find_entropy(df):
    Class = df.keys()[-1]   
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy


def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   
  target_variables = df[Class].unique()  
  variables = df[attribute].unique()    #To give different features in that attribute (like 'Low','High' in Chemistry)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:

        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]


def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None):
    Class = df.keys()[-1]   

    #Building the decision tree:
    
    node = find_winner(df)
    attValue = np.unique(df[node])
    if tree is None:
        tree={}
        tree[node] = {}
    for value in attValue:

        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Pass'],return_counts=True)

        if len(counts)==1:
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) 

    return tree
tree = buildTree(df)
import pprint
pprint.pprint(tree)


{'Physics': {'High': 'Yes',
             'Low': {'Maths': {'High': 'Yes', 'Low': 'No'}},
             'Med': {'Chemistry': {'High': {'Maths': {'High': 'No',
                                                      'Low': 'Yes'}},
                                   'Low': {'Maths': {'High': 'Yes',
                                                     'Low': 'No'}}}}}}


In [20]:
def predict(inst,tree):
    for nodes in tree.keys():

        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0

        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;

    return prediction

In [21]:
inst = df.iloc[7]


In [22]:
inst

Physics      High
Chemistry     Low
Maths         Low
Pass          Yes
Name: 7, dtype: object

In [23]:
data = {'Physics':'Low', 'Chemistry':'Low', 'Maths':'High'}

In [24]:
inst = pd.Series(data)


In [25]:
prediction = predict(inst, tree)
prediction

'Yes'